In [1]:
import scripts.utils.nlp_utils as nlp
import scripts.utils.grammar as gra
import scripts.utils.string_handling as string_hand
import scripts.utils.data_handler as data_hand
import scripts.utils.dscore as dscore
import scripts.utils.meaning as meaning

Loading spacy ...
Finished


# Setup

In [2]:
reference_grammar = gra.read_grammar_and_create_map('referenceGrammar.xml')
diff_grammar = gra.read_grammar_and_create_map('diff_rg_1.xml')
grammar = gra.merge_grammars(reference_grammar, diff_grammar)
prompt_noun_map = meaning.create_meaning_map(grammar)

# Load Data

In [ ]:
#path_to_test_data = "original_data.csv"
# kaldi
path_to_test_data = "data/textProcessing_testKaldi.csv"
path_to_test_data = "data/v11.csv"
path_to_test_data = "data/JJJ_output.csv"
#path_to_test_data = "data/speechrecAlldata.csv"
#path_to_test_data = "data/speechRecAllDataDrive.csv"
#nuance
#path_to_test_data ="data/textProcessing_testNuance.csv"
#path_to_test_data = "data/testDataAnnotated.csv"
speech_rec_data = data_hand.read_test_data(path_to_test_data)

In [ ]:
#grammar['Frag: 4 Tickets nach London']

# NGram

In [ ]:
id_sirlm = {}
with open('languagesolv/v5/id_srilm.csv', 'r') as reader:
    lines = reader.readlines()
    for line in lines:
        split = line.replace("\n", "").split("\t")
        id_ = split[0]
        logprob = float(split[1])
        ppl = float(split[2])
        ppl1 = float(split[3])
        id_sirlm[id_] = {'logprob': logprob, 'ppl':ppl, 'ppl1':ppl1}    

# Language

In [ ]:
classified_prompt = {}
        
def get_judgement(response):
    if response['meaning'] == True and response['language'] == True:
        response['judgement'] = 'accept'

In [ ]:
for prompt_unit in speech_rec_data:
    for response in speech_rec_data[prompt_unit]:
        id_ = response['id']
        #if id_ == '3892' or id_== 'Id' or id_=='3570' or id_ == '3751':
        #    continue
        if id_ == 'Id':
            continue
        if id_ == str(4361) or id_ == str(0) or id_==str(0):
            continue
        #if float(id_sirlm[id_]['ppl1']) <19.3138:
        #if float(id_sirlm[id_]['ppl1']) < 20:
        #if float(id_sirlm[id_]['logprob']) <= -31593.3008: #4.699
        #if float(id_sirlm[id_]['logprob']) <= -31117.5039: #4.699 
        #if float(id_sirlm[id_]['logprob']) <= -28392.2188: #4.699     
        if float(id_sirlm[id_]['logprob']) <= -29645.3398:
        #if float(id_sirlm[id_]['ppl']) <= 33.2845:
        
            response['language'] = True

        #elif float(id_sirlm[id_]['ppl']) < 15.5:
        else:
            response['language'] = False   
        get_judgement(response)
        classified_prompt[response['id']] = response['judgement']          

In [ ]:
score_map, this_score = dscore.score_data(classified_prompt)
dscore.print_scores(this_score)

# Meaning

In [ ]:
def create_id_map():
    id_map = {}
    for prompt_unit in speech_rec_data:
        for response in speech_rec_data[prompt_unit]:
            id_map[response['id']] = response
            
    return id_map

id_map = create_id_map()

def response_in_grammar(response, grammar_responses):
    return response['transcript'] in grammar_responses or response['unique'] in grammar_responses or response['processed'] in grammar

In [ ]:
path_to_reference = "referencetest_final.csv"
reference_id_map = data_hand.read_reference(path_to_reference)

# Reference Grammar

In [ ]:
def apply_reference_grammar(thisgrammar, speech_data):
    for prompt_unit in speech_data:
        if prompt_unit == 'Prompt':
            continue
        for response in speech_data[prompt_unit]: 

            if response_in_grammar(response, thisgrammar[prompt_unit]):
                response['meaning'] = True
                response['language'] = True
            get_judgement(response)
            classified_prompt[response['id']] = response['judgement']          
apply_reference_grammar(grammar, speech_rec_data)

In [ ]:
score_map, this_score = dscore.score_data(classified_prompt)
dscore.print_scores(this_score)

# Cluster

In [ ]:
4.778-3.158

## Credit Card

In [ ]:
import scripts.credit_card as credit_card

In [ ]:
credit_card_map = {}
def apply_credit_card(speech_data):
    credit_card_prompts = credit_card.get_prompts()
    for prompt_unit in credit_card_prompts:
        for response in speech_data[prompt_unit]:            
            if credit_card.accept_credit_card(response['transcript']) or \
                        credit_card.accept_credit_card(response['processed']) or \
                        credit_card.accept_credit_card(response['unique']):  
                response['language'] = True
                
            if meaning.meaning_is_correct(prompt_noun_map, prompt_unit, response['transcript'], response['processed'], response['unique']):
                response['meaning'] = True
            
            get_judgement(response)
            classified_prompt[response['id']] = response['judgement']
            credit_card_map[response['id']] = response['judgement']

apply_credit_card(speech_rec_data)
score_map, this_score = dscore.score_data(credit_card_map)
dscore.print_scores(this_score)
CorrectAccept, GrossFalseAccept, PlainFalseAccept, CorrectReject, FalseReject = dscore.get_single_results(score_map)

## Restarant Cluster


In [ ]:
import scripts.restarant as resta
import scripts.utils.meaning as meaning

In [ ]:
resta.get_prompts()

In [ ]:
restarant_map = {}
def apply_restarant(speech_data):
    restarant_prompts = resta.get_prompts()
    for prompt_unit in restarant_prompts:
        for response in speech_data[prompt_unit]:            
            if resta.accept_restarant(response['transcript']) or \
                resta.accept_restarant(response['processed']) or \
                resta.accept_restarant(response['unique']):  
                response['language'] = True
                
                """
            if response['processed'] == "i would like to pay":
                response['language'] = True
                response['meaning'] = True
                """
            
            if meaning.meaning_is_correct(prompt_noun_map, prompt_unit, response['transcript'], response['processed'], response['unique']):
                response['meaning'] = True
            get_judgement(response)
            
            classified_prompt[response['id']] = response['judgement']
            
            if classified_prompt[response['id']] != response['judgement']:
                print("asdf")
            restarant_map[response['id']] = response['judgement']

apply_restarant(speech_rec_data)
score_map, this_score = dscore.score_data(restarant_map)
dscore.print_scores(this_score)
CorrectAccept, GrossFalseAccept, PlainFalseAccept, CorrectReject, FalseReject = dscore.get_single_results(score_map)

In [ ]:
_, GrossFalseAccept, PlainFalseAccept, _, FalseReject = dscore.get_single_results(score_map)
#analyse_false_classifed_ones(GrossFalseAccept, PlainFalseAccept, FalseReject)

## Zimmer


In [ ]:
import scripts.zimmer as zimmer

In [ ]:
zimmer_map = {}
def apply_zimmer(speech_data):
    zimmer_prompts = zimmer.get_prompts()
    for prompt_unit in zimmer_prompts:
        for response in speech_data[prompt_unit]:            
            if zimmer.accept_zimmer(prompt_unit, response):  
                response['meaning'] = True
            get_judgement(response)
            
            zimmer_map[response['id']] = response['judgement']

apply_zimmer(speech_rec_data)
score_map, this_score = dscore.score_data(zimmer_map)
dscore.print_scores(this_score)

## Capital

In [ ]:
import scripts.capital as capital

In [ ]:
capital_map = {}
def apply_capital(speech_data):
    capital_prompts = capital.get_prompts()
    for prompt_unit in capital_prompts:
        if prompt_unit in speech_data:
            for response in speech_data[prompt_unit]:            
                if capital.accept_capital(prompt_unit, response):  
                    response['meaning'] = True
                get_judgement(response)
                capital_map[response['id']] = response['judgement']
apply_capital(speech_rec_data)
score_map, this_score = dscore.score_data(capital_map)
dscore.print_scores(this_score)

## Ticket

In [ ]:
import scripts.ticket as ticket

In [ ]:
ticket_map = {}
def apply_capital(speech_data):
    ticket_prompts = ticket.get_prompts()
    for prompt_unit in ticket_prompts:
        if prompt_unit in speech_data:
            for response in speech_data[prompt_unit]:            
                if ticket.accept_ticket(prompt_unit, response):  
                    response['meaning'] = True
                get_judgement(response)
                ticket_map [response['id']] = response['judgement']
apply_capital(speech_rec_data)
score_map, this_score = dscore.score_data(ticket_map )
dscore.print_scores(this_score)

In [ ]:
_, GrossFalseAccept, PlainFalseAccept, _, FalseReject = dscore.get_single_results(score_map)
analyse_false_classifed_ones(FalseReject=FalseReject, showFalse=True)

In [ ]:
prompts_done = []
prompts_done.extend(credit_card.get_prompts())
prompts_done.extend(resta.get_prompts())
prompts_done.extend(zimmer.get_prompts())
#prompts_done.extend(ticket.get_prompts())
prompts_done.extend(capital.get_prompts())
prompts_done.remove('Frag: Ich möchte die Dessertkarte')

# Language

In [ ]:
import copy
def insert(tree, key, value):
    #print(key)
    if key:
        first, rest = key[0], key[1:]
        if first not in tree:
            tree[first] = {}
        insert(tree[first], rest, value)
    else:
        tree['key'] = True

tree = {}

def create_tree():
    for prompt_unit in grammar:

        for response in grammar[prompt_unit]:
            tags = nlp.nlp_sentence(response)[2]
            insert(tree, tags, "true")
def existintree(tree, array, rest_tree):
    if len(array) == 0:
        return False
    if array[0] not in rest_tree:
        return False
    if array[0] in rest_tree:
        if 'key' in rest_tree[array[0]] and len(array) == 1:
            return True
    #print(tree[array[0]])
    
    return existintree(tree, array[1:], rest_tree[array[0]])

In [ ]:
false_friends_map = {}
false_friends_map = {"Frag: Ich möchte die Dessertkarte": ['dessert card']}
def false_friends(prompt, sentence):
    if prompt in false_friends_map:
        for false in false_friends_map[prompt]:
            if false in sentence:
                return True
    return False

In [ ]:
def apply_some_magic(thisgrammar, speech_data):
    for prompt_unit in speech_data:
        if prompt_unit == 'Prompt':
            continue
        if prompt_unit in prompts_done:
            continue

        for response in speech_data[prompt_unit]: 
            if meaning.meaning_is_correct(prompt_noun_map, prompt_unit, response['transcript'], response['processed'], response['unique']):
                response['meaning'] = True
            if false_friends(prompt_unit, response['processed']):
                response['meaning'] = False
            
            get_judgement(response)
            
            classified_prompt[response['id']] = response['judgement']          
apply_some_magic(grammar, speech_rec_data)

In [ ]:
score_map, this_score = dscore.score_data(classified_prompt)
dscore.print_scores(this_score)

In [ ]:
#score_map, this_score = dscore.score_data(classified_prompt)
#dscore.print_scores(this_score)

In [ ]:
meaning_and_language_acceptance_rate(speech_rec_data)


In [ ]:
#stop
meaning_and_language_acceptance_rate(speech_rec_data)
#reference_id_map

In [ ]:
_, GrossFalseAccept, PlainFalseAccept, _, FalseReject = dscore.get_single_results(score_map)
analyse_false_classifed_ones(GrossFalseAccept=GrossFalseAccept, showGross=True)

In [ ]:
def get_items(id_,id_entry, reference_entry, typ):
    prompt = reference_entry['prompt']
    
    transcript = data_hand.transcript_processed_unique(reference_entry['transcript'])[0]
    recresult = id_entry['processed']
    
    
    language_their = boolean_to_correct_incorrect(reference_entry['language'])
    language_our = boolean_to_correct_incorrect(id_entry['language'])
    
    
    meaning_their = boolean_to_correct_incorrect(reference_entry['meaning'])
    meaning_our = boolean_to_correct_incorrect(id_entry['meaning'])
    
    return "%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (id_, prompt, transcript, recresult, language_their, language_our, meaning_their, meaning_our, typ)
#ID PROMPT Transcript(Their) RecResult(Our) Language(Their) Language(Our) Meaning(Their) Meaning(Our) Type
def write_to_file(GrossFalseAccept, PlainFalseAccept, FalseReject):
    with open("missclassified_eigenem_wert.csv", 'w') as writer:
        writer.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % ("Id", "Prompt", "Transcript (Their)", "RecResult (Our)", "Language(Their)", "Language(Our)", "Meaning(Their)", "Meaning(Our)", "Type"))
        for id_ in GrossFalseAccept:
            writer.write(get_items(id_, id_map[id_], reference_id_map[id_], "GrossFalseAccept"))

        for id_ in PlainFalseAccept:
            writer.write(get_items(id_, id_map[id_], reference_id_map[id_], "PlainFalseAccept"))

        for id_ in FalseReject:
            writer.write(get_items(id_, id_map[id_], reference_id_map[id_], "FalseReject"))

write_to_file(GrossFalseAccept, PlainFalseAccept, FalseReject)

In [ ]:
GrossFalseAccept

# Analyse

In [ ]:
def boolean_to_correct_incorrect(boolean):
    if boolean:
        return 'correct'
    return 'incorrect'
    
def print_comparision(id_, id_entry, reference_entry):
    print("ID: %s Prompt: %s" % (id_, reference_entry['prompt']))
    print("TRANSCRIPT")
    print("\tour:      %s" % id_entry['processed'])
    print("\ttheir:    %s" % data_hand.transcript_processed_unique(reference_entry['transcript'])[0])
    print("LANGUAGE")
    print("\tour:     %s" % boolean_to_correct_incorrect(id_entry['language']))
    print("\ttheir:   %s" % boolean_to_correct_incorrect(reference_entry['language']))
    print("MEANING")
    print("\tour:     %s" % boolean_to_correct_incorrect(id_entry['meaning']))
    print("\ttheir:   %s" % boolean_to_correct_incorrect(reference_entry['meaning']))
    print("\n")
    
def analyse_false_classifed_ones(GrossFalseAccept=None, PlainFalseAccept=None, FalseReject=None, showGross=False, showPlain=False, showFalse=False):
    if showGross:
        for id_ in GrossFalseAccept:
            print("GrossFalseAccept")
            print_comparision(id_, id_map[id_], reference_id_map[id_])
    if showPlain:
        for id_ in PlainFalseAccept:
            print("PlainFalseAccept")
            print_comparision(id_, id_map[id_], reference_id_map[id_])

    if showFalse:
        for id_ in FalseReject:
            print("FalseReject")
            print_comparision(id_, id_map[id_], reference_id_map[id_])

def meaning_and_language_acceptance_rate(speech_data):
    total = len(reference_id_map)
    meaning = 0
    language = 0
    for prompt_unit in speech_data:
        for response in speech_data[prompt_unit]:
            if response['id'] == 'Id':
                continue
            if response['meaning'] == reference_id_map[response['id']]['meaning']:
                meaning +=1
            if response['language'] == reference_id_map[response['id']]['language']:
                language +=1
    
    print("Meaning: %s" % str(meaning/total*100))
    print("Language: %s" % str(language/total*100))

In [ ]:
reference_id_map['3801']

In [ ]:
id_map['3801']

In [ ]:
PlainFalseAccept

# Reference For Looking UP

In [ ]:
import scripts.utils.dscore as dscore
score_map, this_score = dscore.score_data(classified_prompt)
dscore.print_scores(this_score)

In [ ]:
language_reference_map = {}
meaning_reference_map = {}
with open('referencetest_final.csv','r') as reader:
    lines = reader.readlines()
    for line in lines[1:]:
        split = line.replace("\n", "").split("\t")
        id_ = split[0]
        language = split[4]
        meaning = split[5]
        
        if language == 'incorrect':
            language_reference_map[id_] = False
        else:
            language_reference_map[id_] = True
            
        if meaning == 'incorrect':
            meaning_reference_map[id_] = False
        else:
            meaning_reference_map[id_] = True